In [278]:
"""
Neural Network Model for Airbnb Price Prediction (PyTorch)
"""
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score, mean_squared_error

In [279]:
# Load processed data
train = pd.read_csv('airbnb_train_processed.csv')
test = pd.read_csv('airbnb_test_processed.csv')

In [280]:
X_train = torch.tensor(train.drop('price', axis=1).values, dtype=torch.float32)
y_train = torch.tensor(train['price'].values, dtype=torch.float32)
X_test = torch.tensor(test.drop('price', axis=1).values, dtype=torch.float32)
y_test = torch.tensor(test['price'].values, dtype=torch.float32)

In [281]:
class AirbnbNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze()

model = AirbnbNN(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [282]:
# Training
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.2f}')

Epoch 20, Loss: 189657.31
Epoch 40, Loss: 187822.02
Epoch 60, Loss: 182554.53
Epoch 80, Loss: 171573.94
Epoch 100, Loss: 154257.59


In [283]:
# Evaluation
model.eval()
with torch.no_grad():
    pred = model(X_test)
    r2 = r2_score(y_test.numpy(), pred.numpy())
    rmse = mean_squared_error(y_test.numpy(), pred.numpy()) ** 0.5
    print(f'Neural Network R2: {r2:.4f}, RMSE: ${rmse:.2f}')

Neural Network R2: -0.1879, RMSE: $325.65
